In [1]:
# 01_build_uci_cohort.ipynb
# Build an admission-level dataset from UCI "Diabetes 130-US hospitals" data

import os
import numpy as np
import pandas as pd

RAW_PATH = "../data_raw/diabetic_data.csv"
PROCESSED_PATH = "../data_processed/admissions_features.csv"

print("Raw path:", RAW_PATH)
print("Processed path:", PROCESSED_PATH)

Raw path: ../data_raw/diabetic_data.csv
Processed path: ../data_processed/admissions_features.csv


In [2]:
df_raw = pd.read_csv(RAW_PATH)
print(df_raw.shape)
df_raw.head()

(101766, 50)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [3]:
df = df_raw.copy()

# Drop rows with unknown/invalid gender (they are very few)
df = df[df["gender"].isin(["Male", "Female"])]

# Convert age bins like "[70-80)" to numeric midpoint
age_mid = df["age"].str.extract(r"(\d+)", expand=False).astype(int)
df["age_num"] = age_mid + 5  # midpoint of the 10-year bin

# Binary sex: 1 = female, 0 = male
df["sex_female"] = (df["gender"] == "Female").astype(int)

# Clean race: replace "?" with "Unknown"
df["race_clean"] = df["race"].replace("?", "Unknown")

# One-hot encode race (keeps interpretability + small number of columns)
race_dummies = pd.get_dummies(df["race_clean"], prefix="race")

print("Race dummies:", race_dummies.columns.tolist())
race_dummies.head()

Race dummies: ['race_AfricanAmerican', 'race_Asian', 'race_Caucasian', 'race_Hispanic', 'race_Other', 'race_Unknown']


,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other,race_Unknown
0,False,False,True,False,False,False
1,False,False,True,False,False,False
2,True,False,False,False,False,False
3,False,False,True,False,False,False
4,False,False,True,False,False,False


In [4]:
# Columns that are already numeric but may be read as objects
num_cols = [
    "time_in_hospital",
    "num_lab_procedures",
    "num_procedures",
    "num_medications",
    "number_outpatient",
    "number_emergency",
    "number_inpatient",
    "number_diagnoses",
]

for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df[num_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
time_in_hospital,101763.0,4.396018,2.985092,1.0,2.0,4.0,6.0,14.0
num_lab_procedures,101763.0,43.095909,19.674220,1.0,31.0,44.0,57.0,132.0
num_procedures,101763.0,1.339691,1.705792,0.0,0.0,1.0,2.0,6.0
num_medications,101763.0,16.021835,8.127589,1.0,10.0,15.0,20.0,81.0
number_outpatient,101763.0,0.369368,1.267282,0.0,0.0,0.0,0.0,42.0
number_emergency,101763.0,0.197842,0.930485,0.0,0.0,0.0,0.0,76.0
number_inpatient,101763.0,0.635585,1.262877,0.0,0.0,0.0,1.0,21.0
number_diagnoses,101763.0,7.422649,1.933578,1.0,6.0,8.0,9.0,16.0


In [5]:
diag_cols = ["diag_1", "diag_2", "diag_3"]

# Make everything string and take the first 3 characters as "root"
for c in diag_cols:
    df[c] = df[c].astype(str)

def any_prefix(row, prefixes):
    for c in diag_cols:
        code = row[c]
        if code == "?" or code == "nan":
            continue
        root = code[:3]
        if any(root.startswith(p) for p in prefixes):
            return 1
    return 0

df["comorb_htn"]   = df.apply(any_prefix, axis=1, prefixes=["401", "402", "403", "404", "405"])  # hypertension
df["comorb_heart"] = df.apply(any_prefix, axis=1, prefixes=["428"])                               # heart failure
df["comorb_ckd"]   = df.apply(any_prefix, axis=1, prefixes=["585", "586"])                        # chronic kidney disease
df["comorb_obese"] = df.apply(any_prefix, axis=1, prefixes=["278"])                               # obesity

df[["comorb_htn", "comorb_heart", "comorb_ckd", "comorb_obese"]].mean()

comorb_htn      0.194796
comorb_heart    0.171614
comorb_ckd      0.039140
comorb_obese    0.013030
dtype: float64

In [6]:
ORAL_MEDS = [
    "metformin", "repaglinide", "nateglinide", "chlorpropamide",
    "glimepiride", "acetohexamide", "glipizide", "glyburide",
    "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose",
    "miglitol", "troglitazone", "tolazamide", "examide", "citoglipton",
    "glyburide-metformin", "glipizide-metformin",
    "glimepiride-pioglitazone", "metformin-rosiglitazone",
    "metformin-pioglitazone"
]

ORAL_MEDS = [c for c in ORAL_MEDS if c in df.columns]

# Insulin use: anything except "No"
df["insulin_use"] = (df["insulin"] != "No").astype(int)

# Oral med count: count how many of those columns are not "No"
df["oral_med_count"] = (df[ORAL_MEDS] != "No").sum(axis=1)

df[["num_medications", "insulin_use", "oral_med_count"]].describe().T

,count,mean,std,min,25%,50%,75%,max
num_medications,101763.0,16.021835,8.127589,1.0,10.0,15.0,20.0,81.0
insulin_use,101763.0,0.534408,0.498817,0.0,0.0,1.0,1.0,1.0
oral_med_count,101763.0,0.645293,0.801625,0.0,0.0,0.0,1.0,6.0


In [7]:
df["label"] = (df["readmitted"] == "<30").astype(int)
df["label"].value_counts(normalize=True)
# Binary outcome: 1 if readmitted within 30 days, else 0

label
0    0.888398
1    0.111602
Name: proportion, dtype: float64

In [8]:
id_cols = ["encounter_id", "patient_nbr"]

demo_cols = ["age_num", "sex_female"]

util_cols = [
    "time_in_hospital",
    "num_lab_procedures",
    "num_procedures",
    "num_medications",
    "number_outpatient",
    "number_emergency",
    "number_inpatient",
    "number_diagnoses",
]

comorb_cols = ["comorb_htn", "comorb_heart", "comorb_ckd", "comorb_obese"]

med_cols = ["insulin_use", "oral_med_count"]

# Combine everything
feature_df = pd.concat(
    [
        df[id_cols].reset_index(drop=True),
        df[demo_cols + util_cols + comorb_cols + med_cols].reset_index(drop=True),
        race_dummies.reset_index(drop=True),
        df[["label"]].reset_index(drop=True),
    ],
    axis=1,
)

feature_df.head()
feature_df.shape

(101763, 25)

In [9]:
os.makedirs("../data_processed", exist_ok=True)
feature_df.to_csv(PROCESSED_PATH, index=False)
print("Saved processed dataset to:", PROCESSED_PATH)

Saved processed dataset to: ../data_processed/admissions_features.csv
